# Plotly Dash Season Playoff Dashboard

## Project Description:

The below dashboard allows the user to input NHL team stats and have returned season and playoff outcome predictions.

In [ ]:
# Import packages
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_auth

import numpy as np
import pandas as pd
from pickle import dump
from pickle import load
from textwrap import dedent

import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools

# Set login credentials
USERNAME_PASSWORD_PAIRS = [['data','analyst']]

# establish app
app=dash.Dash(external_stylesheets = [dbc.themes.BOOTSTRAP])

# Set login credentials
auth = dash_auth.BasicAuth(app,USERNAME_PASSWORD_PAIRS)
server = app.server

question1 = "Shooting percentage?, e.g. 10.30"
question2 = "Save percentage?, e.g. 90.02"
question3 = "Average saves per game?, e.g. 25.82"
question4 = "Average shots per game that don't lead to goals?, e.g. 26.91"
question5 = "Percentage of games won when scoring first?, e.g. 75.00"


# Set app layout
app.layout = html.Div([
    
    # Header
    html.Div([html.H2('Season & Playoff Predictions Dashboard',
                      style = {'padding':10,
                               'margin':0,
                               'font-family':'Arial, Helvetica, sans-serif',
                               'background':'#1E90FF',
                               'color':'#FFFFFF',
                               'textAlign':'center'})]),
    
    # Historical data for chosen field
    html.Div([dbc.Row([dbc.Col(dbc.Card(html.Div([html.H3(("Season & Playoff Predictions Form"),
                                                 style = {'padding-bottom': 69.5,
                                                          'padding-top': 10,
                                                          'textAlign':'center'}),
                                         # Question 1
                                         dbc.Row([dbc.Input(id = "input1",
                                                            placeholder = question1,
                                                            type = "number")],
                                                 style = {'padding-bottom': 10,
                                                          'padding-left': 10,
                                                          'padding-right': 10}),
                                         # Question 2
                                         dbc.Row([dbc.Input(id = "input2",
                                                            placeholder = question2,
                                                            type = "number")],
                                                 style = {'padding-bottom': 10,
                                                          'padding-left': 10,
                                                          'padding-right': 10}),
                                         # Question 3
                                         dbc.Row([dbc.Input(id = "input3",
                                                            placeholder = question3,
                                                            type = "number")],
                                                 style = {'padding-bottom': 10,
                                                          'padding-left': 10,
                                                          'padding-right': 10}),
                                         # Question 4
                                         dbc.Row([dbc.Input(id = "input4",
                                                            placeholder = question4,
                                                            type = "number")],
                                                 style = {'padding-bottom': 10,
                                                          'padding-left': 10,
                                                          'padding-right': 10}),
                                         # Question 5
                                         dbc.Row([dbc.Input(id = "input5",
                                                            placeholder = question5,
                                                            type = "number")],
                                                 style = {'padding-bottom': 10,
                                                          'padding-left': 10,
                                                          'padding-right': 10}),
                                         # Submit button
                                         dbc.Row([html.Button('Submit',
                                                              id = 'submit-val')],
                                                 style = {'padding-top': 20,
                                                          'padding-left': 10,
                                                          'padding-right': 10,
                                                          'padding-bottom': 73})]),
                                        body = True,
                                        color = "dark",
                                        outline = True),
                               width = {"size": 5,
                                        "order": 1}),
                       
                       # Above Average Adjusted Wins prediction
                       dbc.Col([dbc.Row([dbc.Card(html.H3(id = 'games',
                                                          style = {'padding': 10,
                                                                   'margin': 0,
                                                                   'font-family': 'Arial, Helvetica, sans-serif',
                                                                   'color': 'black',
                                                                   'textAlign': 'center'}),
                                                 body = True,
                                                 color = "dark",
                                                 outline = True)], 
                                        style = {'padding-bottom': 10,
                                                 'padding-right': 20}),
                               
                               # Gauge
                               dbc.Row([dbc.Card(dcc.Graph(id = 'gauge'),
                                                 body = True, 
                                                 color = "dark", 
                                                 outline = True)],
                                       style = {'padding-right': 20})],
                               width = {"size": 7,
                                        "order": 2})])],
             style = {'padding-left': 20,
                      'padding-right': 20,
                      'padding-top': 40,
                      'padding-bottom': 40}),
    
    # Instructions
    html.Div([html.H1('Instructions',
                      style = {'padding':10,
                               'margin':0,
                               'font-family':'Arial, Helvetica, sans-serif',
                               'background':'#1E90FF',
                               'color':'#FFFFFF',
                               'textAlign':'center'}),
              
              html.Div(dcc.Markdown(dedent('''
              Enter team stats into the Season & Playoff Predictions Form and click submit to view season and 
              playoff predictions. Below are the stats formulas.
              
              * **Shooting Percentage** - Total Goals / Total Shots For
              
              * **Save Percentage** - Total Saves / Total Shots Against
              
              * **Average Saves Per Game** - Total Saves / Total Games Played
              
              * **Average Shots Per Game That Don't Lead to Goals** - (Total Shots / Total Games) * (1 - (Shooting Percentage / 100))
              
              * **Percentage of Games Won When Scoring First** - Total Games Won When Scoring First / Total Games Scored First
              
              * **Above Average Adjusted Wins** - (Total Team Wins + (Total Team Ties / 2)) - ((Total League Wins + (Total League Ties / 2)) / Total Number of Teams)
              ''')),                  
                       style = {'padding':40,
                                'font-family':'Arial, Helvetica, sans-serif',
                                'line-height':30,
                                'textAlign':'left',
                                'fontSize':20})]),
    
    # Final ending block
    html.Div([html.H1('',
                      style = {'padding':30,
                               'margin':0,
                               'font-family':'Arial, Helvetica, sans-serif',
                               'background':'#1E90FF',
                               'color':'#FFFFFF',
                               'textAlign':'center'})])],
    style={'margin':0})

# Return results function
@app.callback([Output('games', 'children'),
               Output('gauge', 'figure')],
              [Input('submit-val', 'n_clicks')],
               state = [State('input1', 'value'),
                        State('input2', 'value'),
                        State('input3', 'value'),
                        State('input4', 'value'),
                        State('input5', 'value')])

def compute(n_clicks, input1, input2, input3, input4, input5):
    
    # Preset inputs
    if input1 == None:
        input1 = 10.30
    if input2 == None:
        input2 = 90.02
    if input3 == None:
        input3 = 25.82
    if input4 == None:
        input4 = 26.91
    if input5 == None:
        input5 = 75.00
    
    # Set binary input
    if input5 > 61:
        input5 = 1
    else:
        input5 = 0
    
    # Load regression model
    model_r = load(open('NHL_Season_Wins_Linear_Regression_Model.pkl', 'rb'))
    
    # Create data frame form input values
    X = [{'shootingPctg': input1,
          'savePctg': input2,
          'savesPerGame': input3,
          'failedShotsPerGame': input4,
          'winScoreFirstGreater61_high': input5}]

    X = pd.DataFrame(X)

    # Predict aboveMeanAdjWins with input values
    y_pred_r = model_r.predict(X)
    # Create data frame with predicted aboveMeanAdjWins values
    y_pred_r = pd.DataFrame(y_pred_r, columns = ['predAboveMeanAdjWins'])
    
    # load logistic regression model
    model_l = load(open('NHL_Playoffs_Logistic_Regression_Model.pkl', 'rb'))

    # Predict with predAboveMeanAdjWins
    y_pred_l = model_l.predict(y_pred_r)
    # Create a data frame from the predicted results
    y_pred_l = pd.DataFrame(y_pred_l, columns = ['predictions'])

    # Creates a data frame from the prediction probabilities
    proba = model_l.predict_proba(y_pred_r)
    proba = pd.DataFrame(proba, columns = ['0', 'predicted_proba'])
    proba['predicted_proba_making_playoffs'] = (1 - proba['predicted_proba']) * 100
    proba = proba[['predicted_proba_making_playoffs']]

    
    # Concatenate both data frames
    results = pd.concat([y_pred_r, proba], axis = 1)
    
    # predAboveMeanAdjWins
    prediction = results.iloc[0,1]

    # Set prediction threshold colour
    if prediction < 50:
        colour = 'red'
    else:
        colour = 'green'

    # Create gauge
    fig = go.Figure(data = go.Indicator(mode = "gauge+number",
                                        value = prediction,
                                        domain = {'x': [0, 1], 'y': [0, 1]},
                                        title = {'text': "Predicted Precentage to Make Playoffs"},
                                        gauge = {'axis': {'range': [0, 100], 'tickwidth': 1, 'tickcolor': "black"},
                                                 'bar': {'color': colour},
                                                 'threshold': {'line': {'color': "black", 'width': 5},
                                                               'thickness': 1,
                                                               'value': 50}}),
                    layout = go.Layout(height = 379))
    
    # predAboveMeanAdjWins prediction via string
    pred_above_mean_adj_wins = "Predicted Above Average Adjusted Wins: " + str(round(results.iloc[0,0],2))

    return pred_above_mean_adj_wins, fig


if __name__ == '__main__':
    app.run_server()